In [152]:
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

In [153]:
df = pd.read_csv('lab04_data.txt')
df

,Tid,Refund,Marital Status,Taxable Income,Evade
0,1,Yes,Single,125K,No
1,2,No,Married,100K,No
2,3,No,Single,70K,No
3,4,Yes,Married,120K,No
4,5,No,Divorced,95K,Yes
5,6,No,Married,60K,No
6,7,Yes,Divorced,220K,No
7,8,No,Single,85K,Yes
8,9,No,Married,75K,No
9,10,No,Single,90K,Yes


In [154]:
df['Taxable Income'] = df['Taxable Income'].str.replace('K','000')
df['Refund'] = df['Refund'].astype('category')
df['Marital Status'] = df['Marital Status'].astype('category')
df['Taxable Income'] = df['Taxable Income'].astype('int64')
df['Evade'] = df['Evade'].astype('category')
df['Refund'] = df['Refund'].cat.codes
df['Marital Status'] = df['Marital Status'].cat.codes
df['Evade'] = df['Evade'].cat.codes
df.set_index('Tid')

,Refund,Marital Status,Taxable Income,Evade
Tid,,,,
1,1,2,125000,0
2,0,1,100000,0
3,0,2,70000,0
4,1,1,120000,0
5,0,0,95000,1
6,0,1,60000,0
7,1,0,220000,0
8,0,2,85000,1
9,0,1,75000,0


In [155]:
df.dtypes

Tid               int64
Refund             int8
Marital Status     int8
Taxable Income    int64
Evade              int8
dtype: object

In [156]:
df['Evade'].value_counts()

0    7
1    3
Name: Evade, dtype: int64

In [157]:
def prob(Ai, Ck):
    return len(Ai)/len(Ck)
def probb(C):
    arr = C.value_counts()
    r = {}
    for i in arr.keys():
        r[i] = (len(C[C==i])/len(C))
    return r

In [158]:
# P(No) & P(Yes)
print(probb(df['Evade']))

{0: 0.7, 1: 0.3}


In [159]:
print('P(Refund=Yes | Yes) : ', end='')
print(prob(df[df['Evade']==1][df['Refund']==1],df[df['Evade']==1]))
print('P(Refund=No  | Yes) : ', end='')
print(prob(df[df['Evade']==1][df['Refund']==0],df[df['Evade']==1]))
print('P(Refund=Yes | No)  : ', end='')
print(prob(df[df['Evade']==0][df['Refund']==1],df[df['Evade']==0]))
print('P(Refund=No  | No)  : ', end='')
print(prob(df[df['Evade']==0][df['Refund']==0],df[df['Evade']==0]))

P(Refund=Yes | Yes) : 0.0
P(Refund=No  | Yes) : 1.0
P(Refund=Yes | No)  : 0.42857142857142855
P(Refund=No  | No)  : 0.5714285714285714


In [160]:
print('P(Status=Divorced | Yes) : ', end='')
print(prob(df[df['Evade']==1][df['Marital Status']==0],df[df['Evade']==1]))
print('P(Status=Married | Yes) : ', end='')
print(prob(df[df['Evade']==1][df['Marital Status']==1],df[df['Evade']==1]))
print('P(Status=Single  | Yes)  : ', end='')
print(prob(df[df['Evade']==1][df['Marital Status']==2],df[df['Evade']==1]))

P(Status=Divorced | Yes) : 0.3333333333333333
P(Status=Married | Yes) : 0.0
P(Status=Single  | Yes)  : 0.6666666666666666


In [161]:
print('P(Status=Divorced | No) : ', end='')
print(prob(df[df['Evade']==0][df['Marital Status']==0],df[df['Evade']==0]))
print('P(Status=Married | No) : ', end='')
print(prob(df[df['Evade']==0][df['Marital Status']==1],df[df['Evade']==0]))
print('P(Status=Single  | No)  : ', end='')
print(prob(df[df['Evade']==0][df['Marital Status']==2],df[df['Evade']==0]))

P(Status=Divorced | No) : 0.14285714285714285
P(Status=Married | No) : 0.5714285714285714
P(Status=Single  | No)  : 0.2857142857142857


In [162]:
def NormalDistribution(Ai, mean, variance):
    return((1/math.sqrt(2*math.pi*variance))*(math.exp(((Ai-mean)**2)/(2*variance))))

In [163]:
mean = df[df['Evade'] == 0]['Taxable Income'].mean()
var = df[df['Evade'] == 0]['Taxable Income'].var()
print('mean: ',mean)
print('variance: ',var)
print('Normal Distribution For(Income=90000, No): ', end='')
print(NormalDistribution(90000, mean, var))

mean:  110000.0
variance:  2975000000.0
Normal Distribution For(Income=90000, No): 7.82281119292927e-06


In [164]:
#X1 = {Refund = Yes, Status = Divorced,Income = 90K, Evade = ?}

# For Evade == Yes
mean = df[df['Evade'] == 1]['Taxable Income'].mean()
var = df[df['Evade'] == 1]['Taxable Income'].var()
# P(Evade = Yes)
evade = probb(df['Evade'])[1]
# P(Refund=Yes | Evade=Yes)
refund = prob(df[df['Evade']==1][df['Refund']==1],df[df['Evade']==1])
# Normal Distribution For(Income==90000, Evade = Yes)
income = NormalDistribution(90000, mean, var)
# P(Status=Divorced | Evade=yes)
status = prob(df[df['Evade']==1][df['Marital Status']==0],df[df['Evade']==1])

evade_yes = (income * evade * refund * status)*100
print('P(Evade == Yes): ',evade_yes,'%')

P(Evade == Yes):  0.0 %


In [165]:
# For Evade == No
mean = df[df['Evade'] == 0]['Taxable Income'].mean()
var = df[df['Evade'] == 0]['Taxable Income'].var()
# P(Evade = No)
evade = probb(df['Evade'])[0]
# P(Refund=Yes | Evade=No)
refund = prob(df[df['Evade']==0][df['Refund']==1],df[df['Evade']==0])
# Normal Distribution For(Income==90000, Evade = No)
income = NormalDistribution(90000, mean, var)
# P(Status=Divorced | Evade=No)
status = prob(df[df['Evade']==0][df['Marital Status']==0],df[df['Evade']==0])

evade_no = (income * evade * refund * status)*100
print('P(Evade == No): ',evade_no,'%')

P(Evade == No):  3.3526333683982576e-05 %


In [166]:
print('{Refund = Yes, Status = Divorced,Income = 90K, Evade = ?}')
if evade_yes > evade_no:
    print('Prediction: (Evade == Yes)')
else:
    print('Prediction: (Evade == No)')

{Refund = Yes, Status = Divorced,Income = 90K, Evade = ?}
Prediction: (Evade == No)


In [167]:
#X2 = {Refund = No, Status = Married,Income = 60K, Evade = ?}
# For Evade == Yes
mean = df[df['Evade'] == 1]['Taxable Income'].mean()
var = df[df['Evade'] == 1]['Taxable Income'].var()
# P(Evade = Yes)
evade = probb(df['Evade'])[1]
# P(Refund=No | Evade=Yes)
refund = prob(df[df['Evade']==1][df['Refund']==0],df[df['Evade']==1])
# Normal Distribution For(Income==60000, Evade = Yes)
income = NormalDistribution(60000, mean, var)
# P(Status=Married | Evade=yes)
status = prob(df[df['Evade']==1][df['Marital Status']==1],df[df['Evade']==1])

evade_yes = (income * evade * refund * status)*100
print('P(Evade == Yes): ',evade_yes,'%')

P(Evade == Yes):  0.0 %


In [168]:
#X2 = {Refund = No, Status = Married,Income = 60K, Evade = ?}
# For Evade == No
mean = df[df['Evade'] == 0]['Taxable Income'].mean()
var = df[df['Evade'] == 0]['Taxable Income'].var()
# P(Evade = No)
evade = probb(df['Evade'])[0]
# P(Refund=No | Evade=No)
refund = prob(df[df['Evade']==0][df['Refund']==0],df[df['Evade']==0])
# Normal Distribution For(Income==60000, Evade = No)
income = NormalDistribution(60000, mean, var)
# P(Status=Married | Evade=No)
status = prob(df[df['Evade']==0][df['Marital Status']==1],df[df['Evade']==0])

evade_no = (income * evade * refund * status)*100
print('P(Evade == No): ',evade_no,'%')

P(Evade == No):  0.0002544867623238046 %


In [169]:
print('{Refund = No, Status = Married,Income = 60K, Evade = ?}')
if evade_yes > evade_no:
    print('Prediction: (Evade == Yes)')
else:
    print('Prediction: (Evade == No)')

{Refund = No, Status = Married,Income = 60K, Evade = ?}
Prediction: (Evade == No)
